In [ ]:
import json
import os
import sys
import math
import argparse
import numpy as np
from scipy.special import erf
from scipy.interpolate import interp1d
from mpl_toolkits.mplot3d import Axes3D
from shutil import rmtree, copy
from matplotlib import pyplot as plt
from matplotlib import gridspec, cm
from matplotlib import patches as mpatches 

plt.style.use('publication_lhy11009')

R = 8.314

# directory to the aspect Lab
ASPECT_LAB_DIR = os.environ['ASPECT_LAB_DIR']
RESULT_DIR = os.path.join(ASPECT_LAB_DIR, 'results')

sys.path.append(os.path.join(ASPECT_LAB_DIR))
import shilofue.PlotDepthAverage as PDAver
import shilofue.FlowLaws as FlowLaws
import shilofue.ParsePrm as ParsePrm
import shilofue.ThermalModel as TModel
import shilofue.Rheology as Rheology

# import utilities in subdirectiory
sys.path.append(os.path.join(ASPECT_LAB_DIR, 'utilities', "python_scripts"))
import Utilities


In [ ]:
# read the parameters from HK 03.
# water is in the water fugacity
# rheology = "HK03"
rheology = "WarrenHansen23"

rheology_prm_dict = Rheology.RHEOLOGY_PRM()
diffusion_creep = getattr(rheology_prm_dict, rheology + "_diff")
dislocation_creep = getattr(rheology_prm_dict, rheology + "_disl")
rheology_dict = {'diffusion': diffusion_creep, 'dislocation': dislocation_creep}

print("diffusion_creep: ", diffusion_creep)
print("dislocation_creep: ", dislocation_creep)

In [ ]:
creep = diffusion_creep
# 1.9e-6
diff_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': 0.0, 'V': -2.1e-6}
disl_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': 0.0, 'V': 0.0}
# creep_correction = {'A': 1.0, 'p': 0.0, 'r': 0.0, 'n': 0.0, 'E': 75e3, 'V': 5e-06}

# fit new rheology
stress_ref = 50.0 # MPa
# strain_rate_ref = 1e-5
P_ref = 100.0e6 # Pa
T_ref = 1250.0 + 273.15 # K
Coh_ref = 1000.0 # H / 10^6 Si
d_ref = 15.0 # mu m

strain_rate_diff_correction = Rheology.CreepStrainRate(diff_correction, stress_ref, P_ref, T_ref, d_ref, Coh_ref)
strain_rate_disl_correction = Rheology.CreepStrainRate(disl_correction, stress_ref, P_ref, T_ref, d_ref, Coh_ref)

# screen output
print("strain_rate_diff_correction:", strain_rate_diff_correction) 
print("strain_rate_disl_correction:", strain_rate_disl_correction) 

# apply the correction
diffusion_creep['E'] += diff_correction['E']
diffusion_creep['V'] += diff_correction['V']
dislocation_creep['E'] += disl_correction['E']
dislocation_creep['V'] += disl_correction['V']

# screen output
print("diffusion_creep: ", diffusion_creep)
print("dislocation_creep: ", dislocation_creep)

In [ ]:
Operator = Rheology.RHEOLOGY_OPR()

source_dir = os.path.join(ASPECT_LAB_DIR, 'tests', 'integration', 'fixtures', 'test_rheology')
da_file = os.path.join(source_dir, "depth_average.txt")
Operator.ReadProfile(da_file)

fig_path = os.path.join(ASPECT_LAB_DIR, "results", "%s" % (rheology)) + ".png"

Operator.MantleRheology(assign_rheology=True, diffusion_creep=diffusion_creep, dislocation_creep=dislocation_creep, save_profile=1,\
        use_effective_strain_rate=True, save_json=False, fig_path=fig_path, strain_rate_diff_correction=strain_rate_diff_correction,\
        strain_rate_disl_correction=strain_rate_disl_correction)